In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# APIs - Práctica Guiada III

<a id="section_toc"></a> 
## Tabla de Contenidos

[1- Intro](#section_intro)

$\hspace{.5cm}$[1.1- La librería requests](#section_requests)

[2- Formando un DataFrame](#section_df)

$\hspace{.5cm}$[2.1- Parseando un JSON con Pandas](#section_json_normalize)

$\hspace{.5cm}$[2.2- Data Wrangling](#section_data_wrangling)

[3- Paginado: cómo armar el DataFrame final](#section_paginado)

---

<a id="section_intro"></a> 
###  1- Intro
[volver a TOC](#section_toc)

El objetivo de esta práctica guiada es generar el dataset que utilizaremos en el ejercicio de la clase. Para ello, utilizaremos funcionalidades un poco más avanzadas de la librería `requests`, la cual tiene una reputación notable como una de las librerías más "Pythonicas" que existen.

<a id="section_requests"></a> 
#### 1.1- La librería requests
[volver a TOC](#section_toc)

Como explicamos en la clase teórica, el protocolo HTTP nos permite emplear distintas "acciones" por medio de las requests HTTP. La librería requests es un paquete de Python que contiene métodos sencillos y lineales para ejecutar estas Requests (GET, POST, PUT, DELETE).

Cuando una API no tiene una librería desarrollada para consumirse (o incluso cuando la tiene pero nos resulta incómoda o restrictiva), la librería requests nos permite directamente traer los resultados de ejecutar una HTTP request al endpoint que deseemos, siempre y cuando tengamos la posibilidad de acceder a él (lógicamente no es muy sencillo intentar mandar una DELETE request a la API de Mercadolibre, por ejemplo, a menos que seamos los propietarios de una determinada publicación). 

Para traernos datos, lo más habitual, y lo que usaremos en esta práctica, es una request tipo GET. Si esta request es exitosa, nos traerá un mensaje (hoy en día lo más común es el formato JSON) con la información de la página que pedimos ver. A continuación, veamos un ejemplo sencillo:

In [ ]:
from datetime import datetime
import requests
import json
import pandas as pd

url = "https://api.mercadolibre.com/sites/MLA/search?q=nintendo+switch&condition=new&limit=50"  # Endpoint a consultar
response = requests.get(url)

La respuesta a una request es un objeto tipo `requests.models.Response`. Este tipo de objetos contiene, además del texto del mensaje de respuesta, un código de status. Podemos ver en esta imagen el significado de distintos códigos de respuesta:

<img src='./img/statuscode2.jpg' align='center'/>

Para una request de tipo GET, un código 200 significa que todo está bien. Veamos qué status obtenemos para nuestra consulta a Mercadolibre:

In [ ]:
print("Status Code:", response.status_code)

Veamos ahora el encabezado de la respuesta. De aquí podemos obtener varios datos interesantes, como cuándo fue hecha la request y qué encoding tiene el mensaje que nos devuelve en la respuesta.

In [ ]:
print(response.headers)

---

En el atributo `response.text` de la respuesta, obtenemos el contenido del mensaje. Por ser esta una request tipo get, pueden hacer la prueba de ingresar a la URL de la request en su navegador, y observarán el json del resultado que aquí obtenemos en forma programática.

El texto viene como un string, pero queremos poder tratarlo como una estructura de datos, con lo cual vamos a parsear el json con la librería `json` de Python. Esta librería viene con cualquier instalación de Python, por lo cual no requiere setup adicional. `json.loads()` convertirá la respuesta en objetos nativos de Python (diccionarios y listas), con el fin de poder armar nuestro dataset.

En el caso particular de la respuesta de Mercadolibre, dentro del json encontramos una clave "results", en la cual encontraremos toda publicación perteneciente a la respuesta. Vamos a quedarnos con los resultados en nuestra variable `results`.

In [ ]:
data = json.loads(response.text)

results=data["results"]

# print(results[0])

<a id="section_df"></a> 
###  2- Formando un DataFrame
[volver a TOC](#section_toc)

Para el ejercicio de esta clase, vamos a querer usar un DataFrame de publicaciones de Mercadolibre con el fin de predecir la cantidad de unidades vendidas. Para ello, queremos formar un DataFrame de pandas y formar un CSV. Usar el constructor de DataFrame sobre nuestra variable `results` tiene varias limitaciones muy serias, que iremos resolviendo conforme avancemos en la práctica, vamos a ver algunas de ellas a continuación.

In [ ]:
df=pd.DataFrame(results)
df.head()

En este primer resultado podemos observar varios de los problemas de los que hablábamos anteriormente:
* Muchas de nuestras columnas son diccionarios. ¿Cómo podemos acceder a los campos en cada uno?
* Sólo tenemos 50 registros. ¿Qué pasa si modificamos el parámetro `limit` en nuestra request?
* Muchas de las publicaciones que vemos no refieren al producto que buscamos, sino que nos hablan de accesorios para el mismo. ¿Es posible filtrar estas respuestas?

<a id="section_json_normalize"></a> 
#### 2.1- Parseando un JSON con Pandas
[volver a TOC](#section_toc)

Afortunadamente, como dijimos antes, el formato JSON se ha vuelto un estándar ampliamente difundido en los respuestas de APIs y, como tal, Pandas tiene su propia implementación para leer información de JSONs y expandir los subcampos.

Por todo lo susodicho, la función `json_normalize` de Pandas resuelve en forma sencilla y completa el primero de los tres problemas que teníamos anteriormente. Veamos cómo queda ese DataFrame:

In [ ]:
df_expand=pd.json_normalize(results)
df_expand.head()

Podemos ver que nuestro nuevo DataFrame tiene muchas más columnas. Comparemos ambos índices:

In [ ]:
print("DataFrame original:", list(df.columns))
print()
print("DataFrame expandido:", list(df_expand.columns))
print()

Sin embargo, el DataFrame expandido trae dos problemas adicionales:
1. Genera demasiadas columnas, ya que no todos los subcampos nos son relevantes.
1. Muestra los niveles de expansión por medio de puntos, lo cual nos molestaría para la notación de atributos (por ejemplo `df_expand.seller.seller_reputation.real_level` nos arrojaría un error, en lugar de devolver la serie `df_expand["seller.seller_reputation.real_level"]`.

Estos dos problemas se resuelven sencillamente, renombrando las columnas y luego quedándonos sólo con las relevantes. Ensallaremos una solución para estos problemas, que luego reproduciremos en nuestro dataset final.

<a id="section_data_wrangling"></a> 
#### 2.2- Data Wrangling
[volver a TOC](#section_toc)

Antes de generar un DataFrame de un tamaño importante para efectuar predicciones, queremos dejar armadas funciones para poder limpiarlo en forma rápida. Comenzaremos renombrando columnas para remover esos puntos molestos.

In [ ]:
def rename_json_cols(df):
    df = df.copy()
    df.columns = [col.replace(".", "__") for col in df.columns]
    return df

In [ ]:
df_no_dots = rename_json_cols(df_expand)
df_no_dots.head()

En primer lugar removeremos las columnas geográficas, con todos los subelementos correspondientes, así como los ids innecesarios, y las variables categóricas que sólo tengan un valor posible.

In [ ]:
def drop_columns(df, drop_geo=True, drop_ids=True, drop_uniques=True):
    """
    Función para eliminar columnas innecesarias del DataFrame.
    
    Parameters
    ----------
    
    drop_geo : bool
        Booleano para definir si eliminamos las veriables geográficas. Default, True.
    drop_ids: bool
        Booleano para definir si eliminamos las veriables de IDs y similares. Default, True.
    drop_uniques: bool
        Booleano para definir si eliminamos las veriables con un solo valor posible. Default, True.

    """
    df = df.copy()
    # Variables geográficas:
    if drop_geo:
        nogeo_mask = df.columns[
            ~df.columns.str.contains(r"^address__|^seller_address__|^seller__eshop")
        ]
        df = df[nogeo_mask]
    if drop_uniques:
        series_uniques = df.apply(lambda x: len(x.value_counts()) != 1).values
        df = df.loc[:, series_uniques]
    if drop_ids:
        for col in ["permalink", "seller__permalink", "catalog_product_id",
                    "thumbnail", "attributes", "original_price", "official_store_id"]:
            try:
                df = df.drop(col, axis=1)
            except Exception:
                print(f"{col} ya fue eliminada en chequeos anteriores.")
        series_ids = df.apply(lambda x: len(x.value_counts()) != len(df)).values
        df = df.loc[:, series_ids]
    # También columnas excesivamente nulas.
    series_nulls = df.isnull().mean() < 0.6
    df = df.loc[:, series_nulls]

    
    return df

In [ ]:
df_pruned = drop_columns(df_no_dots)

In [ ]:
df_pruned.head()

Con esto nos quedamos con un df bastante terminado, pero tenemos dos columnas que nos resultan interesantes por ser fechas: `seller__registration_date` y `stop_time`. Queremos convertir la primera en años de actividad del vendedor, y la segunda en días restantes a la publicación.

In [ ]:
def parse_dates(df):
    """Convierte 'stop_time' en 'days_remaining' y 'seller__registration_date' en 'years_active'."""
    df = df.copy()
    df["days_remaining"] = (
        pd.to_datetime(df['stop_time'], utc=True) - pd.Timestamp(datetime.utcnow(), tz="UTC")
    ).dt.days
    df["years_active"] = (
        pd.Timestamp(datetime.utcnow(), tz="UTC") -  pd.to_datetime(df['seller__registration_date'], utc=True)
    ).dt.days // 365
    df = df.drop(["stop_time", "seller__registration_date"], axis=1)
    return df

In [ ]:
df_parsed = parse_dates(df_pruned)
df_parsed.head()

Ya tenemos un DataFrame cuyo formato parece apto para predecir, ahora intentemos obtener un dataframe de mayor tamaño, y enfocado sólo en el tipo de producto que buscamos.

<a id="section_paginado"></a> 
###  3- Paginado: cómo armar el DataFrame final
[volver a TOC](#section_toc)

En esta última sección vamos a resolver los dos problemas que nos quedan: el reducido tamaño del dataset, y los productos "accesorios" al que realmente estamos buscando. Para esto usaremos el dataset a escala que generamos antes, con el fin de determinar a qué categoría pertenece en la API de Mercadolibre nuestro producto buscado, y efectuaremos una request con paginado, para obtener algunos registros más.

In [ ]:
# Obtengamos primero la categoría.
df_expand.domain_id.value_counts()

Podemos observar que la categoría que corresponde al producto es también la más usual. Sin embargo, corresponde a un poco más de la mitad de los registros de la respuesta. Como tenemos un límite a cuántos datos podemos pedir, lo más inteligente es incluir ese filtro en la request misma. Utilicemos ahora la categoría (`category_id`), con el fin de obtener un código que sirva como parámetro en la API:

In [ ]:
print(df_expand.category_id.value_counts())
category = df_expand.category_id.value_counts().index[0]

In [ ]:
url = "https://api.mercadolibre.com/sites/MLA/search?"
params = {
    "q": "nintendo switch".replace(" ", "+"),
    "limit": 50,
    "condition": "new",
    "category": category
}
# q=nintendo+switch&condition=new&limit=50&category=MLA438566  # Endpoint a consultar
response = requests.get(url, params = params)

In [ ]:
pd.DataFrame(json.loads(response.text)["results"]).domain_id.value_counts()

Esta parametrización logra traernos sólo los datos que buscamos, con lo que sólo nos queda resolver el problema del paginado. Vamos a armar una solución que vaya de la request al dataset finalizado, con el fin de poder generar un dataset del tema que a ustedes les interese:

**NOTA IMPORTANTE:** la API tiene un límite de requests máximas permitidas, así que no ejecuten esta celda muchas veces seguidas.

In [ ]:
def full_request(search="iphone 11", limit=50, total_obs=2000, save=True, category=None, access_token=None):
    url = "https://api.mercadolibre.com/sites/MLA/search?"
    results = []
    if category is None:
        params = {
            "q": search.replace(" ", "+"),
            "limit": 50,
            "condition": "new",
        }
        response_ = requests.get(url, params=params)
        cats_ = pd.DataFrame(json.loads(response_.text)["results"]).category_id.value_counts()
        category = cats_.index[0]
        del cats_, response_
    params["category"] = category
    params["limit"] = limit
    params["access_token"] = access_token
    for offset in range(0, total_obs, limit):  # Paginamos usando el parámetro
        try:
            params["offset"] = offset
            response = requests.get(url, params=params)
            results += response.json()["results"]
        except KeyError:
            print(
                "Error: El máximo de datos permitidos fue superado o no puede encontrarse más publicaciones.",
                f"Se obtuvieron {len(results)} publicaciones."
            )
            break
    df = pd.json_normalize(results)
    df = rename_json_cols(df)
    df = drop_columns(df)
    df = parse_dates(df)
    if save:
        df.to_csv(f'../Data/{search.lower().replace(" ", "_")}_meli.csv', index=False)
    return df

**NOTA IMPORTANTE:** Crear la carpeta "Data" en el directorio de la clase si aún no la tienen. 

In [ ]:
final_df = full_request(total_obs=1200)

In [ ]:
print(final_df.shape)
final_df.head()

Antes de intentar resolver un problema predictivo sobre la cantidad de ventas, veamos cómo se distribuyen los precios.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')

final_df.price[final_df.price < final_df.price.quantile(.99)].plot.kde()
plt.xlabel("Price");